# **ПРОГНОЗ СТОИМОСТИ АВТОМОБИЛЕЙ**
# EDA
## Задача
Необходимо создать модель, которая будет предсказывать стоимость автомобиля по его характеристикам. Для оценки использовать метрику MAPE

С помощью данного ноутбука мы провели обаботку и анализ данных и подготовили датасет к обучению.

Также в этом проекте мы использовали:

* Ноутбук, через который парсили https://www.kaggle.com/tatianamukhidaeva/prj5-kirill-n-tanya-m-parser
* Спарсенный датасет https://www.kaggle.com/tatianamukhidaeva/cars-df-v2
* Ноутбук, в котором провели EDA и обучение https://www.kaggle.com/tatianamukhidaeva/prj5-kirill-n-tanya-m-eda

## Библиотеки

Задача: спрогнозировать стоимость автомобилей, используя данные 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor, Pool
from sklearn.ensemble import StackingRegressor

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

import time
import os

from pandas import Series
import re

import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from tqdm import tqdm
from datetime import timedelta, datetime, date

import json

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# Подготовим функции для простоты дальнейшей работы:
def visualizing_number(data, column): 
    bins = 100
    if data[column].nunique() < 100:
        bins = data[column].nunique()
    
    fig, axes = plt.subplots(2,1, figsize=(26,8))
    sns.boxplot(ax=axes[0], y = data[column],data=data, orient='h')
    axes[0].set_title(column)
    data[column].hist(ax=axes[1], bins = bins)
    plt.show()
    print()
    

def get_outliers(column): 
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    min_out = Q1 - 1.5 * IQR
    max_out = Q3 + 1.5 * IQR
    return (column < min_out).sum() + (column > max_out).sum(), min_out, max_out

def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [ ]:
VERSION    = 16
DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' 
DIR_TRAIN2 = '../input/cars-df-v2/' #импортируем данные, которые были предвариельно взяты с сайта авто.ру
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [ ]:
!ls '../input'

In [ ]:
train_2020 = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv')
train = pd.read_csv(DIR_TRAIN2+'cars_df.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [ ]:
train_2020.info()

In [ ]:
train.info()

In [ ]:
test.info()

Заметим, что в файле 2020 года объявлений гораздо больше, чем в датасете 2021 года. Однако спарсить большее количество не удалось. Также цены в 2021 году сильно изменились. Учтем инфляцию и объединим датасеты 2020 и 2021 года.

## Учет инфляции

С 2020 по 2021 год цены на автомобили сильно поднялись, особенно на популярные модели. Почитать об этом можно по ссылкам:
- https://mag.auto.ru/article/risepricescalltouchpres/
- https://cenamashin.ru/statistika/moskva/avg_price?seg=1

Причем цены на разные бренды менялись по-разному

In [ ]:

# удалим знак валюты и пробелы в цене
train['price'] =train['price'].astype(str)
train['price'] = train['price'].apply(lambda x: x if len(x) ==1 else ''.join(x.split()[0:-1])).astype(int)

In [ ]:
# инфляция по брендам


brands = test.brand.unique()
infs = []
for brand in brands:
    mean20 = train_2020[train_2020["brand"]==brand].price.mean()
    mean21 = train[train["brand"]==brand].price.mean()
    inf = (mean21-mean20)/mean20
    infs.append(inf)
    print(f'{brand} 2020: {mean20}')
    print(f'{brand} 2021: {mean21}')
    print(f'Инфляция: {round(inf * 100,2)}%')
    print()


Учтем инфляцию и изменим цены так, чтобы они были похожи на осень 2020 года

In [ ]:
brands = test.brand.unique()
tt = train.copy()
display(tt.head(2))
for brand in brands:
    mean20 = train_2020[train_2020["brand"]==brand].price.mean()
    mean21 = train[train["brand"]==brand].price.mean()
    inf = (mean21-mean20)/mean20
    if inf>0:
        tt.loc[tt["brand"]==brand, ['price']] = tt.loc[tt["brand"]==brand, ['price']].apply(lambda x: round(x/(1+inf),0))
display(tt.head(2))

In [ ]:
# Цены изменились верно.
data = tt

## Data Preprocessing

### Подготовка столбцов к объединению

In [ ]:
print(train.columns)
print(train_2020.columns)
print(test.columns)

In [ ]:
# Создадим список общих переменных для теста и трейна:
s2020 = list(train_2020.columns)
s2021 = list(train.columns)
s = set(list(test.columns))
no2020 = [x for x in s if not x in s2020] #найдем стобцы, которых нет в трейне
print('Столбцы, которые есть в тесте, но их нет в трейне 2020:', no2020)
no2021 = [x for x in s if not x in s2021] #найдем стобцы, которых нет в трейне
print('Столбцы, которые есть в тесте, но их нет в трейне 2021:', no2021)
#посмотрим, какие признаки отличают два датасета, чтобы определить нужно ли дополнительно парсить данные для трейна или не стоит
#test[s3]
#model_info дублирует полезную информацию из столбца car_url, 
#super_gen - аккумулирует и дублирует много информации с разных стобцов,
#а остальные столбцы имеют много пропусков и не совсем репрезентативны


**Столбцы, которых нет в трейне 2021**

'super_gen'  - аккумулирует и дублирует много информации с разных стобцов, 

'model_info' - дублирует полезную информацию из столбца car_url,

'Владение' - длительность последнего  или общего владения (не удалось спарсить, так как эти данные подгружаются AJAX), 

'vendor' - продавец, пока уберем этот столбец, 

'equipment_dict' - аналог 'equipmentGroups', 

'complectation_dict' - частично содержится в 'equipmentGroups'

In [ ]:
train['equipment_dict'] = train['equipmentGroups']
train = train.drop(['equipmentGroups'], axis=1)

train.columns

**Столбцы, которых нет в трейне 2020**

'sell_id' - создадим его, наполнив 0

'car_url' - создадим, изменив model

'priceCurrency', 'parsing_unixtime', 'complectation_dict', 'equipment_dict', 'image', 'super_gen' - можно удалить,


In [ ]:
train_2020['sell_id'] = 0
train['sell_id'] = 0
train_2020['car_url'] = train_2020['model']

In [ ]:
# список общих переменных
columns2021 = [x for x in s2021 if not x in no2021]
print('Общие для 2021')
print('Общее количество', len(columns2021))
print(columns2021)

columns2020 = [x for x in s2020 if not x in no2020]
print('Общие для 2020')
print('Общее количество', len(columns2020))
print(columns2020)
# columns

Удалим столбцы:

Из 2021

- *не несут информации для цены* - 'image','parsing_unixtime',
- *слишком сложные в обработке* - 'description', 'equipmentGroups'
- *дублируют другие признаки* -'model_name', 
- *единственное значение* - 'priceCurrency', 'Состояние'


In [ ]:
print('train - priceCurrency - ', train.priceCurrency.value_counts(), sep='\n')
print('test - priceCurrency - ', test.priceCurrency.value_counts(), sep='\n')
print()
print('train - Состояние - ', train['Состояние'].value_counts(), sep='\n')
print('test - Состояние - ', test['Состояние'].value_counts(), sep='\n')

In [ ]:
to_del = ['image','parsing_unixtime','priceCurrency','description','model_name', 'Состояние','equipmentGroups']

In [ ]:
columns = [x for x in columns2021 if not x in to_del] 
columns = ([x for x in columns if x in columns2020+['car_url', 'sell_id']] )

У тестового датасета нет цены, добавим нулевую цену

In [ ]:
test['price']=0

In [ ]:
df_train_2020 = train_2020[columns]
df_train = train[columns]
df_test = test[columns]

### Объединение train и test

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train_2020['sample'] = 1 # помечаем где у нас трейн
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).append(df_train_2020, sort=False).reset_index(drop=True) # объединяем
data[data['sample']==0].shape

In [ ]:
data.info()

У нас получилось 21 признак. Из которых 6 числовые, 15 категориальные.

In [ ]:
#переименуем признаки
data.rename(columns={'productionDate': 'production_date',
                     'vehicleConfiguration': 'vehicle_configuration',
                     'vehicleTransmission': 'vehicle_transmission',
                     'name':'gear',
                     'Владельцы': 'owners_qty',
                     'Владение': 'ownership_time',
                     'ПТС': 'licence',
                     'Привод': 'type_of_drive',
                     'Руль': 'steering_wheel',
                     'Таможня': 'customs', 
                     'Price': 'price',
                     'bodyType': 'body_type', 
                     'engineDisplacement': 'engine_volume',
                     'enginePower': 'engine_power',
                     'fuelType': 'fuel_type',
                     'modelDate': 'model_date',
                     'numberOfDoors': 'number_of_doors',}, inplace=True)
data.columns

* body_type - тип кузова,
* brand - марка автомобиля,
* car_url - url страницы объявления
* color - цвет автомобиля,
* engine_displacement - объём двигателя,
* engine_volume - мощность двигателя,
* fuel_type - тип топлива,
* mileage - пробег,
* model_date - дата релиза модели,
* name - имя, введенное пользователем
* number_of_doors - количество дверей,
* production_date - дата производства автомобиля
* vehicle_configuration - конфигурация транспортного средства (ТС),
* vehicle_transmoission - тип коробки передач,
* owners_qty - количество владельцев,
* licence - паспорт ТС,
* type_of_drive - тип привода,
* steering_wheel - сторона руля,
* customs - этап растаможки,
* price - цена автомобиля, целевой параметр,
* sample - индикатор принадлежности данных к тесту (0) и трейну (1),


In [ ]:
#удалим дубликаты из датафрейма
# data[data.duplicated(keep=False)]
data = data.drop_duplicates()
data[data['sample'] ==0].info()

# # ***EDA***

In [ ]:
# числовые признаки
num_cols = []

# бинарные признаки
bin_cols = []

# категориальные признаки
cat_cols = []

# обработанные категориальные признаки
ready_cat_cols = []

## **CAR_URL**

In [ ]:
data1 = data.copy() # сохраним данные в data1, чтобы зафиксировать изменения
data[data['sample']==0].shape

In [ ]:
data.car_url

In [ ]:
#Изменим столбец car_url на модель авто

value = data.car_url.apply(lambda x: x.lower() if len(x.split('/'))==1 else x.split('/')[7].lower())
idx = data.columns.get_loc('brand') 
data.insert(loc=idx+1, column='model_of_car', value=value )
data = data.drop('car_url',1)
data[data['sample'] ==0].info()

In [ ]:
data['model_of_car'].value_counts

In [ ]:
cat_cols.append('model_of_car')


# **bodyType**

In [ ]:
data['body_type'].unique()

In [ ]:
#переделаем названия
data['body_type'] = data['body_type'].astype(str).apply(lambda x: None if x.strip()=='' else x)
data['body_type'] = data['body_type'].apply(lambda x: x.split(' ')[0].lower())

In [ ]:
sns.countplot(y = data['body_type'], data = data, order = data['body_type'].value_counts().index)
# sns.countplot?

Самый популярный кузов внедорожник, на втором месте седан

посмотрим на пропуски,если они есть,заполним их кузов типа "внедорожник"

In [ ]:
data.at[93550,'body_type'] = None

In [ ]:
data['body_type'].isna().sum()

In [ ]:
data['body_type'] = data['body_type'].fillna(data['body_type'].mode())

In [ ]:
cat_cols.append('body_type')

In [ ]:
data[data['sample']==0].shape

# **brand**

In [ ]:
data['brand'].unique() 

In [ ]:
data['brand'].value_counts().plot.barh()

Наиболее популярные бренды-, БМВ, Mercedes

In [ ]:
data['brand'].isnull().sum()

In [ ]:
cat_cols.append('brand')

# **color**

In [ ]:
data['color'].unique() 

In [ ]:
data['color'].value_counts().plot.barh()

Самый популярные цвета среди автомобилей - чёрный и белый

In [ ]:
#переименуем признак
color_dict = {'040001':'black', 
              'FAFBFB':'white',
              'CACECB':'silver',
              '97948F':'grey',
              'чёрный':'black',
              '0000CC':'blue',
              'белый':'white',
              '200204':'brown',
              'EE1D19':'red',
              'серебристый':'silver',
              'серый':'grey',
              'синий':'blue',
              '007F00':'green',
              'C49648':'beige',
              'красный':'red', 
              'коричневый':'brown',
              '22A0F8':'light_blue',
              'зелёный':'green',
              '660099':'purple',
              'DEA522':'gold',
              '4A2197':'violet',
              'бежевый':'beige',
              'FFD600':'yellow',
              'голубой':'light_blue',
              'FF8649':'orange',
              'золотистый':'gold',
              'пурпурный':'purple',
              'фиолетовый':'violet',
              'жёлтый':'yellow',
              'оранжевый':'orange',
              'FFC0CB':'pink',
              'розовый':'pink'
             }

In [ ]:
data['color'] = data['color'].map(color_dict)

Разделим цвета по популярности. Самые популярные: 1, менее популярные: 2,еще менее популярные: 3, не популярные: 4

In [ ]:
  
pop_color_dict = {
    'white': 1,
    'black': 1,
    'blue': 2,
    'silver': 2,
    'grey': 2,
    'brown': 3,
    'red': 3, 
    'green': 3,
    'beige': 3,
    'light_blue': 3,
    'purple': 4,
    'yellow': 4, 
    'orange': 4,
    'violet': 4, 
    'gold': 4, 
    'pink': 4
}
data['color'] = data['color'].map(pop_color_dict)

In [ ]:
ready_cat_cols.append('color')

In [ ]:
data['color'].isnull().sum()


In [ ]:
data['color'].fillna(1, inplace=True)

In [ ]:
data[data['sample']==0].shape

# **fuelType**

In [ ]:
data['fuel_type'].unique()

In [ ]:
data['fuel_type'].value_counts().plot.barh()

Наиболее популярные машины на бензине, наименее - на газу

In [ ]:
# дадим латинские названия
d = {
    'бензин': 'petrol', 
    'дизель': 'diesel', 
    'гибрид': 'hybrid', 
    'электро': 'electro', 
    'газ': 'gas'
}
data['fuel_type'] = data['fuel_type'].map(d)

In [ ]:
cat_cols.append('fuel_type')

# **model_date,production_date**

In [ ]:
data2 = data.copy() # Зафиксируем датасет перед изменениями

In [ ]:
np.sort(data['model_date'].unique())

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(20,5))
plt.title('Год выпуска автомобиля')
plt.xticks(rotation=90)
sns.countplot(x='model_date', data=data, orient='v')

In [ ]:
data['model_date'][data['model_date']< 1980].value_counts(dropna=False).sort_index(ascending=False)

Автомобилей, произведенных ранее 1980 года очень мало. Однако должны ли мы уметь предсказывать цену на них? Возможно должны. Но:
- во-первых: данных не достаточно,
- во-вторых: даже если модель будет плохо предсказывать цены на столь старые автомобили, для основной массы результат предсказания не изменится, или даже станет лучше.

Вывод: примем эти автомобили за выбросы

In [ ]:
# примем  автомобили старше 1980 г. за выбросы
data = data[~((data.model_date < 1980)&(data.sample==1))]

In [ ]:
data[data['sample']==0].shape

In [ ]:
data['model_date'].isnull().sum()

In [ ]:
data = data.dropna(subset=['model_date'])

In [ ]:
data['production_date'].isnull().sum()

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(20,5))
plt.title('Количество моделей, выпущенных в году')
plt.xticks(rotation=90)
sns.countplot(x='model_date', data=data, orient='v')

In [ ]:
#введем новые признаки возраст модели и возраст машины
from datetime import date

today = date.today()
d1 = int(today.strftime("%Y"))

data['model_date'] = data['model_date'].astype('int')
idx = data.columns.get_loc('model_date') 
data.insert(loc=idx+1, column='model_d', value=d1 - data['model_date'] )
data.insert(loc=idx+1, column='production_d', value=d1 - data['production_date'] )


In [ ]:
num_cols.append('model_date')

In [ ]:
num_cols.append('production_date')

In [ ]:
num_cols.append('model_d')

In [ ]:
num_cols.append('production_d')

In [ ]:
data[data['sample']==0].shape

# **Gear**

In [ ]:
data3 = data.copy() # Зафиксируем датасет перед изменениями
# data = data3.copy()

In [ ]:
data['gear'].unique()

In [ ]:
#из признака выберем только тип трансмиссии, т. к. остальные данные у нас есть
data['gear'] = data['gear'].astype(str)
idx = data.columns.get_loc('color') 
data.insert(loc=idx+1, column='transmis', value= data['gear'].str.extract('([A][T]|[M][T]|[A][M][T]|[C][V][T])',
                                                expand=False).str.strip())

In [ ]:
sns.countplot(x = data['transmis'], data = data)

Очевидно, что наиболее популряной коробкой является AT

In [ ]:
data['transmis'].isnull().sum()

In [ ]:
data['transmis']=data['transmis'].fillna('AT')

In [ ]:
cat_cols.append('transmis')

In [ ]:
data = data.drop('gear', 1)

# **number_of_doors**

In [ ]:
data4 = data.copy() # Зафиксируем датасет перед изменениями

In [ ]:
data['number_of_doors'].unique()

In [ ]:
mode = round(data['number_of_doors'].mean(),0)
mode

In [ ]:
data['number_of_doors']=data['number_of_doors'].fillna(mode)

In [ ]:
sns.countplot(x = data['number_of_doors'], data = data)

Наибольшее количество автомобилей имеет 5 дверей

In [ ]:
num_cols.append('number_of_doors')

In [ ]:
data[data['sample']==0].shape

# **vehicle_configuration**

In [ ]:
data5 = data.copy() # Зафиксируем датасет перед изменениями

In [ ]:
data['vehicle_configuration'].value_counts()

Получим данные о типе коробки передач

In [ ]:
data['vehicle_configuration'] = data['vehicle_configuration'].astype(
                str).apply(lambda x: x if len(x) == 1 else x.split())

In [ ]:
data['vehicle_configuration'] = data['vehicle_configuration'].apply(
    lambda x: x[0].lower() if len(x) == 1 else x[1].lower())

In [ ]:
data['vehicle_configuration'].value_counts().plot.barh()

Больше всего машин с автоматической коробкой передач

In [ ]:
cat_cols.append('vehicle_configuration')

# **engine_volume**

In [ ]:
data6 = data.copy() # Зафиксируем датасет перед изменениями
# data = data6.copy()

In [ ]:
data['engine_volume'].value_counts(dropna=False).head(50)

In [ ]:
data['engine_volume'] = data['engine_volume'].astype(str)

Получим объем двигателя

In [ ]:
data['engine_volume'] = data['engine_volume'].str.extract('(\d.\d)',expand=False).str.strip()

In [ ]:
data['engine_volume']=data['engine_volume'].astype(float)

In [ ]:
data['engine_volume'] = data['engine_volume'].apply(lambda x: round(x/100,1) if (x>10) else round(x,1))

In [ ]:
mean = round(data['engine_volume'].mean(),1)
mean

In [ ]:
data[data['sample']==0]['engine_volume'].isnull().sum()

In [ ]:
data['engine_volume'].fillna(mean, inplace=True)

In [ ]:
data['engine_volume'].value_counts().plot.barh()

In [ ]:
num_cols.append('engine_volume')

In [ ]:
data[data['sample']==0].shape

# **engine_power**

In [ ]:
data7 = data.copy() # Зафиксируем датасет перед изменениями

In [ ]:
data['engine_power']

In [ ]:
data['engine_power'] = data['engine_power'].astype(str).apply(lambda x: x.split()[0])
data['engine_power'] = data['engine_power'].astype(float)

In [ ]:
data['engine_power'].value_counts()

In [ ]:
data['engine_power'].value_counts().sort_index()

In [ ]:
# Разобьём признак на категории
def engine_power(x):
    if x < 100: x = 1
    elif 99 < x < 150: x = 2
    elif 149 < x < 200: x = 3
    elif 199 < x < 250: x = 4
    elif 249 < x < 300: x = 5
    elif 299 < x < 350: x = 6
    elif 349 < x < 400: x = 7
    elif 399 < x < 450: x = 8
    elif 449 < x < 500: x = 9
    elif 499 < x < 550: x = 10
    elif 549 < x < 600: x = 11
    else: x = 12
    return x  

In [ ]:
data['engine_power'] = data['engine_power'].map(engine_power)

In [ ]:
plt.figure(figsize=(16,8))
sns.countplot(x = data['engine_power'], data = data) 
plt.title('Мощность двигателя в л.с.')

In [ ]:
data['engine_power'].isnull().sum()

In [ ]:
num_cols.append('engine_power')

In [ ]:
data[data['sample']==0].shape

# **mileage**

In [ ]:
data8 = data.copy() # Зафиксируем датасет перед изменениями
# data = data8.copy()

In [ ]:
data['mileage']

In [ ]:
data['mileage'] = data['mileage'].astype(str).apply(lambda x: x.replace('\xa0',''))
data['mileage'] = data['mileage'].astype(str).apply(lambda x: x.replace('км',''))
data['mileage'] = data['mileage'].apply(lambda x: int(x.replace(' ','')))
data['mileage']

In [ ]:
data['mileage'].isna().sum()

In [ ]:
num_cols.append('mileage')

# **mileage_per_year**

In [ ]:
#Добавим новый признак - годовой пробег 
import math
value= round(data['mileage'].astype(int)
                                                       /(data['model_d'].apply(lambda x: x if x>0 else x+1)),-2)
idx = data.columns.get_loc('licence') 
data.insert(loc=idx+1, column='mileage_per_year', value= value)
data['mileage_per_year'] = data['mileage_per_year'].apply(lambda x: x if x != math.inf else 0 )

In [ ]:
plt.figure(figsize=(26,8))
plt.title('Пробег в год')
plt.xticks(rotation=90)

plot_ = sns.countplot(x = data['mileage_per_year'], data = data) 
for ind, label in enumerate(plot_.get_xticklabels()):
    if ind % 10 == 0:  # every 10th label is kept
        label.set_visible(True)
    else:
        label.set_visible(False)

In [ ]:
num_cols.append('mileage_per_year')

# **type_of_drive**

In [ ]:
data9 = data.copy() # Зафиксируем датасет перед изменениями
# data = data9.copy()

In [ ]:
data['type_of_drive'].unique()

In [ ]:
data['type_of_drive'].value_counts(dropna=False)

In [ ]:
# дадим латинские названия
d = {
    'передний': 'front', 
    'полный': 'full', 
    'задний': 'back'
}
data['type_of_drive'] = data['type_of_drive'].map(d)

In [ ]:
data['type_of_drive']=data['type_of_drive'].fillna('front')

In [ ]:
plt.figure(figsize=(8,8))
sns.countplot(x = data['type_of_drive'], data = data)
plt.title('Тип привода')

Наибольшее ко-во машин имеет передний привод

In [ ]:
cat_cols.append('type_of_drive')

# **steering_wheel**

In [ ]:
data['steering_wheel'].value_counts(dropna=False)

In [ ]:
steering_wheel_dict = {'LEFT':'left', 
                       'Левый':'left',
                       'RIGHT':'right', 
                       'Правый':'right',
                        }
data['steering_wheel'] = data['steering_wheel'].map(steering_wheel_dict)

In [ ]:
plt.figure(figsize=(8,8))
sns.countplot(x = data['steering_wheel'], data = data) 

Наибольшее количество машин леворульные

In [ ]:
data['steering_wheel'].isnull().sum()

In [ ]:
bin_cols.append('steering_wheel')

In [ ]:
data[data['sample']==0].shape

# **owners_qty**

In [ ]:
data11 = data.copy() # Зафиксируем датасет перед изменениями
# data = data11.copy()

In [ ]:
data['owners_qty'].value_counts(dropna=False)
# data['owners_qty'].unique()

In [ ]:
data['owners_qty'] = data['owners_qty'].astype(
                str).apply(lambda x: x if len(x) == 1 else x.split())
data['owners_qty']

In [ ]:
data['owners_qty']= data['owners_qty'].apply(
    lambda x: x[0].lower() if len(x) == 1 else x[0].lower())

In [ ]:
data['owners_qty']=data['owners_qty'].astype(float)

In [ ]:
data['owners_qty'].isnull().sum()

In [ ]:
mode = round(data['owners_qty'].mode(),0)
mode

In [ ]:
data['owners_qty'].fillna(3, inplace=True)

In [ ]:
ready_cat_cols.append('owners_qty')


# **licence**

In [ ]:
data12 = data.copy() # Зафиксируем датасет перед изменениями
# data = data12.copy()

In [ ]:
data['licence'].unique()

In [ ]:
licence_dict={'Оригинал': 1,
             'Дубликат': 0,
             'ORIGINAL':1,
             'DUPLICATE':0}
data['licence'] = data['licence'].map(licence_dict)

In [ ]:
data['licence'].value_counts( dropna=False)

Если лицензия не указана, то не будем ставить, что она есть, возможно машина без документов. Установим значение 2 - отличается от остальных

In [ ]:
data['licence'] = data['licence'].fillna(2)

In [ ]:
ready_cat_cols.append('licence')

# **customs**

In [ ]:
data['customs'].value_counts()

всего один нерастаможенный автомобиль. Нет смысла держать этот признак в датасете

In [ ]:
data = data.drop('customs', 1)

## **vehicle_transmission**

In [ ]:
data['vehicle_transmission'].value_counts()

Такой признак уже был

In [ ]:
data = data.drop('vehicle_transmission',1 )

## Price

Уже делали обработку при учете инфляции для train 2021

In [ ]:
data['price']

In [ ]:
data['price'][(data['sample']==1)&(data['price'].isna())]

Так как это целевая переменная, удалим пустые значения

In [ ]:
data.dropna(subset=['price'], inplace=True)

In [ ]:
# Посмотрим, есть ли  дубликаты:
print('Количество дубликатов:',len(data) - len(data.drop_duplicates()))

In [ ]:
# data[data.duplicated(keep=False)]
data = data.drop_duplicates()
data.isna().sum()

# **Анализ признаков**

In [ ]:
data15 = data.copy()
# data = data15.copy()

In [ ]:
print(bin_cols)
print(num_cols)
print(cat_cols)
print(ready_cat_cols)

In [ ]:
data.info()

## 1. Анализ числовых переменных

In [ ]:
# посмотрим на корреляцию признаков между собой
plt.figure(figsize=(8, 5))
sns.heatmap(data[num_cols + ['price']].corr().abs(), vmin=0, vmax=1,
            annot=True, fmt=".2f", cmap="YlGnBu")

Признаки model_date, production_date, model_d, production_d полностью коррелируют друг  другом, поэтому оставим только один из них: production_d

In [ ]:
data = data.drop(['model_date', 'production_date', 'model_d'], 1)

In [ ]:
num_cols.remove('model_date')
num_cols.remove('production_date')
num_cols.remove('model_d')

In [ ]:
for col in num_cols:
    visualizing_number(data, col)

In [ ]:
# #определяем значимость наших переменных:
imp_cat = Series(mutual_info_classif(data[data['price'] >0][num_cols], 
                                     data[data['price'] >0]['price'],
                                     discrete_features = True), index = num_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')
plt.title('Значимость чиловых переменных для price')

**imp_cat - это весы для признаков, будем использовать эти данные при обучении**

## 2. Анализ категориальных и бинарных переменных

In [ ]:
data[ready_cat_cols + cat_cols + bin_cols]

Заметим, что vehicle_configuration и transmis означают одно и то же. Удалим vehicle_configuration

In [ ]:
data = data.drop(['vehicle_configuration'], 1)

In [ ]:
cat_cols.remove('vehicle_configuration')

In [ ]:
for col in cat_cols:
    print(col)
    print(data[col].value_counts())
    print()

Разделим cat_cols:
- Если категорий мало, то будем обрабатывать их с помощью Dumming,
- Если категорий много, то - LabelEncoder 

К категориям на LabelEncoder добавим bin_cols - бинарные признаки

In [ ]:
data[bin_cols]

In [ ]:
# cat_cols
cat_oh_cols = ['fuel_type', 'transmis', 'type_of_drive']
cat_le_cols = ['model_of_car', 'body_type', 'brand', 'steering_wheel']

In [ ]:
# Преобразуем все значения категориальных признаков в числа:
data = pd.get_dummies(data, columns=cat_oh_cols, dummy_na=False)

In [ ]:
# Преобразуем все значения категориальных признаков в числа:

label_encoder = LabelEncoder()
for i in cat_le_cols:
    data[i] = label_encoder.fit_transform(data[i])
data.info()    

**Оценим влияние категориальных признаков на целевую переменную**

In [ ]:
cat_new_cols = data.columns
# len(cat_cols)
cat_new_cols = [x for x in cat_new_cols if x not in (num_cols + ['sell_id','price','sample'])]
data[cat_new_cols].info()

In [ ]:
imp_num = imp_cat.copy()

In [ ]:
imp_cat = Series(mutual_info_classif(data[data['price'].isna() == False][cat_new_cols], 
                                     data[data['price'].isna() == False]['price'],
                                     discrete_features = True), index = cat_new_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')
plt.title('Влияние категориальных признаков на price')

Объединим веса в один сериес. Нужно перевести его в list, но чтобы значения признаков стояли в том же порядке, что и в data

In [ ]:
imp_weights = pd.concat([imp_num, imp_cat]).drop_duplicates()
cols = data.columns
weights = [imp_weights[col] for col in cols if col not in ['sell_id', 'price', 'sample']]
weights


# Улучшение числовых признаков
Проведем нормализацию и логарифмирование

In [ ]:
# # data = data20.copy()
# dn = data.copy()

К числовым признакам добавим еще и те категориальные признаки, которые мы обработали с помощью LabelEncoding, так как получили в них большой разброс чисел

In [ ]:
# dd = dn.drop(['sample','sell_id','price'],1).copy()
# dd = pd.DataFrame(StandardScaler().fit_transform(dd), columns = dd.columns)
# dn[dd.columns] = dd
# dn

In [ ]:
# # 'production_d', 'engine_power', 'mileage', 'mileage_per_year'
# dn[['production_d', 'engine_power', 'mileage', 'mileage_per_year']] = np.log(dn[['production_d', 'engine_power', 'mileage', 'mileage_per_year']] + 1)

In [ ]:
# for col in num_cols:
#     visualizing_number(data20, col)

In [ ]:
# for col in num_cols:
#     visualizing_number(dn, col)

In [ ]:
# data20 = data.copy()
# data = dn.copy()

# Разделение переменных

In [ ]:
X = data.query('sample == 1').drop(['sample', 'price', 'sell_id'], axis=1)
X_sub = data.query('sample == 0').drop(['sample', 'price'], axis=1)
y = data.query('sample == 1')['price'].values
X_sub

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)
X_train

In [ ]:
# def standart():



# # Model 1 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Fit

In [ ]:
# model = CatBoostRegressor(iterations = 5000,
#                           random_seed = RANDOM_SEED,
#                           eval_metric='MAPE',
#                           custom_metric=['R2', 'MAE'],
#                           silent=True,
#                          )

stsc = StandardScaler()
X_train = stsc.fit_transform(X_train)
X_test = stsc.transform(X_test)

# Добавим веса
X_train = X_train * weights
X_test = X_test * weights


In [ ]:
# model.fit(X_train, y_train,
#          #cat_features=cat_features_ids,
#          eval_set=(X_test, y_test),
#          verbose_eval=0,
#          use_best_model=True,
#          #plot=True
#          )

# # model.save_model('catboost_single_model_baseline.model')

# # оцениваем точность
# predict = model.predict(X_test)
# print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")
# # Без стандартизации Точность модели по метрике MAPE: 15.61%

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**

In [ ]:
np.log(y_train)

In [ ]:
# model = CatBoostRegressor(iterations = 5000,
#                           random_seed = RANDOM_SEED,
#                           eval_metric='MAPE',
#                           custom_metric=['R2', 'MAE'],
#                           silent=True,
#                          )

# model.fit(X_train, np.log(y_train),
#          #cat_features=cat_features_ids,
#          eval_set=(X_test, np.log(y_test)),
#          verbose_eval=0,
#          use_best_model=True,
#          #plot=True
#          )

# # model.save_model('catboost_single_model_2_baseline.model')

In [ ]:
# predict_test = np.exp(model.predict(X_test))
# predict_submission = np.exp(model.predict(X_sub))

In [ ]:
# print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")
# # без стандартизации и логорифмирования
# # Точность модели по метрике MAPE: 13.74%

# # Model 2 : LinearRegression

In [ ]:
# lin_reg = LinearRegression().fit(X_train, y_train)
# y_pred = lin_reg.predict(X_test)
# print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")

# # Точность модели по метрике MAPE: 117.26%

# # Model 3: GradientBoosting

In [ ]:
# # max_depth должно примерно соответствовать количеству признаков или быть больше.
# gb = GradientBoostingRegressor(min_samples_split=2,
#                                learning_rate=0.03,
#                                max_depth=30,
#                                n_estimators=1000)
# gb.fit(X_train,y_train)
# # Сначала попробуем без логорифмирования таргета
# y_pred = gb.predict(X_test)
# print(f"Точность модели по метрике MAPE_без лого: {(mape(y_test, y_pred))*100:0.2f}%")

# # 
# # Точность модели по метрике MAPE: 14.96%
# # 

# # без стандартизации и логорифмирования 15.65%

In [ ]:
# #Тоже самое с логорифмом таргета
# gb.fit(X_train,np.log(y_train))
# y_pred = gb.predict(X_test)
# print(f"Точность модели по метрике MAPE c лого: {(mape(y_test, y_pred))*100:0.2f}%")
# # Точность модели по метрике MAPE c лого: 100.00%

# # Model 4: Forest

In [ ]:
# rf = RandomForestRegressor(n_estimators=1000,
#                            n_jobs=-1,
#                            max_depth=30,
#                            max_features='log2',
#                            random_state=RANDOM_SEED,
#                            oob_score=True)
# rf.fit(X_train,y_train)



In [ ]:
# # Сначала попробуем без логорифмирования таргета
# y_pred = rf.predict(X_test)
# print(f"Точность модели по метрике MAPE_без лого: {(mape(y_test, y_pred))*100:0.2f}%")

# # Точность модели по метрике MAPE_без лого: 18.15%, max_depth=50, || 100
# # Точность модели по метрике MAPE_без лого: 18.10%, max_depth=30

# # Model 5: Xgboosting

In [ ]:
import xgboost as xgb
xb = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.03, \
                      max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000)
xb.fit(X_train, np.log(y_train+1))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(xb.predict(X_test))))*100:0.2f}%")


#Точность модели по метрике MAPE: 14.15%,  max_depth=8
#Точность модели по метрике MAPE: 13.89%,  max_depth=12
#Точность модели по метрике MAPE: 14.07%,  max_depth=35
#Точность модели по метрике MAPE: 14.02%,  max_depth=20

# # Model 6: StackingRegressor

Выбирем для стека самые лучшие модели,  которые мы получили: Xgboosting, CatBoosting. Для обработки результатов возьмем линейную регрессию.
До этого пробовали брать за последнюю модель CatBoosting, но результаты оказались хуже.

Брать много сложных моделей не будем, так как даже при 2-3 моделях выполнение обучения идет более 30 минут.

In [ ]:
estimators = [('xb',xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.03, \
                      max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000)),
              ('xb2',xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.03, \
                      max_depth=12, alpha=1, n_jobs=-1, n_estimators=500)),
              ('cb', CatBoostRegressor(iterations = 2000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         ))
            ]



In [ ]:
st_ensemble = StackingRegressor(estimators=estimators,final_estimator=LinearRegression())

# оцениваем точность
st_ensemble.fit(X_train, np.log(y_train))
predict_e = np.exp(st_ensemble.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_e))*100:0.2f}%")

# Точность модели по метрике MAPE: 14.01%: xb1_12_1000, xb2_12_500,-> cb
# Точность модели по метрике MAPE: 13.80%: xb1_12_1000, xb2_12_500, cb -> LR

# Выбор модели


Во время исследования все модели запускались много раз. У нас были следующие параметры для проверки:
- Подключать или нет датасет 2020 года
- учитывать или нет инфляцию для датасета 2021 года
- учитывать или нет авто старше 2020 года в трейне,
- логорифмировать или нет числовые признаки,
- Выбор обработки категориальных данных: Dumming, LabelEncoding
- Проводить или нет стандартизацию данных перед обучением
- Учитывать вес каждого признака или нет.

По логике все эти процедуры должны улучшать обучение, однако, например 

- учет или не учет авто старше 2020 года не изменяет результат. Просто таких авто мало в процентах.

- Подключение датасета 2020 года повлияло очень сильно. Метрика на XBoost улучшилась в submission с 20% до 13%. Однако на трейне получили 13% в обоих случаях. Это конечно же влияет то, что цены за год сильно изменились.

- Логорифмирование числовых данных немного ухудшает модель. Убрали эту процедуру.

- Выбор того, как распределить категориальные переменные для обработки указан выше. Это оказался оптимальный вариант, который дал не так много новых столбцов.


<table class="table">
  <thead>
    <tr>
      <th scope="col">#</th>
      <th scope="col">Model</th>
      <th scope="col">отброшены старше 1980</th>
      <th scope="col">Проведена стандартизация и веса</th>
      <th scope="col">max_depth</th>
      <th scope="col">MAPE</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th scope="row">1</th>
      <td>CatBoost</td>
      <td>no</td>
      <td>yes</td>
      <td>-</td>
      <td>17.15%</td>
    </tr>
    <tr>
      <th scope="row">1</th>
      <td>CatBoost (log res)</td>
      <td>no</td>
      <td>yes</td>
      <td>-</td>
      <td>14.24%</td>
    </tr>
    <tr>
      <th scope="row">2</th>
      <td>LinearRegression</td>
      <td>no</td>
      <td>yes</td>
      <td>-</td>
      <td>117.26%</td>
    </tr>
    <tr>
      <th scope="row">3</th>
      <td>GradientBoosting</td>
      <td>no</td>
      <td>yes</td>
      <td>30</td>
      <td>14.96%</td>
    </tr>
    <tr>
      <th scope="row">4</th>
      <td>RandomForest</td>
      <td>no</td>
      <td>yes</td>
      <td>30</td>
      <td>18.10%</td>
    </tr>
    <tr>
      <th scope="row">4</th>
      <td>RandomForest</td>
      <td>no</td>
      <td>yes</td>
      <td>50</td>
      <td>18.15%</td>
    </tr>
    <tr>
      <th scope="row">5</th>
      <td>Xgboosting</td>
      <td>no</td>
      <td>yes</td>
      <td>12</td>
      <td>13.89%</td>
    </tr>
      <tr>
      <th scope="row">5</th>
      <td>Xgboosting</td>
      <td>no</td>
      <td>yes</td>
      <td>20</td>
      <td>14.02%</td>
    </tr>
  <tr>
      <th scope="row">5</th>
      <td>Xgboosting</td>
      <td>no</td>
      <td>yes</td>
      <td>35</td>
      <td>14.07%</td>
    </tr>
    <tr>
      <th scope="row">6</th>
      <td>Stacking</td>
      <td>no</td>
      <td>yes</td>
      <td>Stack models: Xb,Xb, Cb. Finish: LR</td>
      <td>13.80</td>
    </tr>
  </tbody>
</table>

# Submission

In [ ]:
X_sub_st = stsc.transform(X_sub.drop('sell_id',1))
X_sub_st = X_sub_st * weights
# X_sub_st['sell_id']


In [ ]:
# xb.fit(X_train, np.log(y_train+1))
# X_train
pred = np.exp(st_ensemble.predict(X_sub_st))

In [ ]:
predict_submission = np.round(pred,-3).astype('int')


In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_2_v{VERSION}.csv', index=False)
sample_submission.head(10)

In [ ]:
!ls '../input'

# Выводы:

Лучший результат на валидации показала модель Xgboosting: 13.69623%, лучше чем StackingRegression: 13.77556%
Как мы уже не раз убеждались, сильнее всего на результат влияют сами данные. Спарсить удалось только 34 тыс. объявлений 2021 года, а в тесте данные 2020 года. Цены на авто сильно изменились, причем не равномерно по моделям. Сделали учет инфляции в среднем по брендам и добавили датасет объявлений за 2020 год.

В Exploratory data analysis провели следующие преобразования:
- выделили модели из car_url,
- выделили группы цветов по-популярности,
- рассчитали пробег в год,
- рассчитали возраст авто,
- обработали категориальные переменные.

Что можно было еще сделать:
- Можно было обработать поля комплектации equipmentGroups, обычно от этого цена сильно зависит.
